#### Hessenberg decomposition

A Hessenberg matrix is a special kind of square matrix, one that is "almost" triangular

To be exact, an upper Hessenberg matrix has zero entries below the first subdiagonal, and a lower Hessenberg matrix has zero entries above the first superdiagonal

Similar to QR, Hessenberg decomposition aims to find for $A\in \mathbf{R}^{m \times m}$, orthogonal matrix $Q$ such that

$$A=QHQ^T$$

where $H$ is in Hessenberg form

#### Arnoldi iteration

The Arnoldi iteration performs exactly like modified Gram-Schmidt for progressively finding both first $(n+1)$ vectors in $Q$ and the $(n+1) \times n$ upper-left section of $H$

To see this, we can write more explicitly a `reduced version` of $AQ=QH$

$$A\begin{bmatrix}q_1 & q_2 & \cdots q_n\end{bmatrix}=\begin{bmatrix}q_1 & q_2 & \cdots q_{n+1}\end{bmatrix}\begin{bmatrix}h_{11} & \cdots & h_{1n} \\ h_{21}& & \vdots \\
& \ddots & \vdots \\ & & h_{n+1,n}\end{bmatrix}$$

The reason we have $n+1$ on the right is that for each $Aq_n$, the computation requires $q_{n+1}$ and $h_{n+1,n}$ due to the non-zeros in the first subdiagonal (We can see this from the full $AQ=QH$)

Different from MGS where for each $a_n$ only one new $q_n$ is produced, for Arnoldi, we will use the residual to compute $q_{n+1}$ and $h_{n+1, n}$

#### Example

In [24]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [25]:
def arnoldi_iteration(A, b, n, tol=1e-10):
    m = A.shape[0]
    Q = np.zeros((m, n+1))
    H = np.zeros((n+1, n))
    Q[:, 0] = b / np.linalg.norm(b)

    for i in range(n):
        v = A @ Q[:, i]

        for j in range(i+1):
            H[j, i] = Q[:, j] @ v
            v -= H[j, i] * Q[:, j]

        H[i+1, i] = np.linalg.norm(v)
        if H[i+1, i] < tol:
            break
        Q[:, i+1] = v / H[i+1, i]

    return Q, H

In [26]:
np.random.seed(42)

m = 10
A = np.random.rand(m, m)
b = np.random.rand(m)

Q, H = arnoldi_iteration(A, b, 6)

In [27]:
print('Q:\n', Q)
print('\nCheck orthogonality in Q:\n', Q.T @ Q)
print('\nH:\n', H)

print('\nAQ:\n', A @ Q[:,:-1])
print('\nQH:\n', Q @ H)
print('\nDifference:\n', np.linalg.norm(A @ Q[:,:-1] - Q @ H))

Q:
 [[ 0.0201  0.5472  0.0083 -0.2643  0.0899 -0.5132 -0.1788]
 [ 0.4069 -0.0498 -0.3796 -0.3602  0.0081  0.1151 -0.3134]
 [ 0.2010  0.1566  0.1453  0.5647  0.6900  0.0674  0.0034]
 [ 0.3251  0.1234  0.1668  0.0822 -0.0178 -0.1264 -0.6818]
 [ 0.5802 -0.2723 -0.2154  0.0397  0.1663 -0.3238  0.3365]
 [ 0.1594  0.3828  0.2770 -0.0451 -0.0558  0.6176 -0.0255]
 [ 0.2624  0.1207  0.2998 -0.5883  0.2736  0.1485  0.3787]
 [ 0.4830 -0.1455  0.1083  0.2360 -0.4806  0.1801  0.0106]
 [ 0.1463  0.3159  0.3667  0.1840 -0.4075 -0.3669  0.3090]
 [ 0.0492  0.5496 -0.6691  0.1881 -0.1133  0.1659  0.2282]]

Check orthogonality in Q:
 [[ 1.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000]
 [ 0.0000  1.0000  0.0000 -0.0000 -0.0000 -0.0000  0.0000]
 [ 0.0000  0.0000  1.0000 -0.0000 -0.0000  0.0000  0.0000]
 [ 0.0000 -0.0000 -0.0000  1.0000 -0.0000 -0.0000 -0.0000]
 [ 0.0000 -0.0000 -0.0000 -0.0000  1.0000 -0.0000  0.0000]
 [ 0.0000 -0.0000  0.0000 -0.0000 -0.0000  1.0000  0.0000]
 [ 0.0000  0.0000  0.00